In [15]:
from ultralytics import YOLO
import json
import numpy as np

In [5]:
model = YOLO('yolo11s.pt')

results = model("detecton_box_test/0a0a0b1a-7c39d841.jpg")
results


image 1/1 /Users/beaulin/DS 340 Project/Analysis/road_object_detec/detecton_box_test/0a0a0b1a-7c39d841.jpg: 384x640 2 cars, 3 trucks, 58.5ms
Speed: 1.3ms preprocess, 58.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

In [ ]:
with open("detecton_box_test/0a0a0b1a-7c39d841.json", "r") as f:
    gt_data = json.load(f)

print(gt_data)

{'name': '0a0a0b1a-7c39d841', 'frames': [{'timestamp': 10000, 'objects': [{'category': 'car', 'id': 0, 'attributes': {'occluded': True, 'truncated': False, 'trafficLightColor': 'none'}, 'box2d': {'x1': 555.647397, 'y1': 304.228432, 'x2': 574.015906, 'y2': 316.474104}}, {'category': 'car', 'id': 1, 'attributes': {'occluded': True, 'truncated': False, 'trafficLightColor': 'none'}, 'box2d': {'x1': 554.116689, 'y1': 318.004813, 'x2': 567.89307, 'y2': 328.719775}}, {'category': 'car', 'id': 2, 'attributes': {'occluded': True, 'truncated': False, 'trafficLightColor': 'none'}, 'box2d': {'x1': 523.502507, 'y1': 311.881977, 'x2': 544.932435, 'y2': 327.189068}}, {'category': 'car', 'id': 3, 'attributes': {'occluded': True, 'truncated': False, 'trafficLightColor': 'none'}, 'box2d': {'x1': 492.888326, 'y1': 311.881977, 'x2': 526.563926, 'y2': 337.904031}}, {'category': 'truck', 'id': 4, 'attributes': {'occluded': True, 'truncated': False, 'trafficLightColor': 'none'}, 'box2d': {'x1': 393.392235, '

In [43]:
gt_xyxy = np.array([obj.get('box2d', {0, 0, 0, 0}) for obj in gt_data['frames'][0]['objects']])   # shape: (G, 4)
gt_classes = [obj["category"] for obj in gt_data['frames'][0]['objects']]

In [58]:

gt_boxes_list = []
gt_classes_list = []
for obj in gt_data['frames'][0]['objects']:
    if 'box2d' in obj:
        box = obj['box2d']
        gt_boxes_list.append([box['x1'], box['y1'], box['x2'], box['y2']])
        gt_classes_list.append(obj["category"])

gt_xyxy = np.array(gt_boxes_list, dtype=int)
gt_classes = gt_classes_list

print(f"Found {len(gt_xyxy)} ground truth boxes.")
print(f"Found {len(gt_classes)} ground truth classes.")
print(gt_classes)

class_to_id = {
    "car": 0, 
    "truck": 1,
    "traffic light": 2,
    "traffic sign": 3,
    "person": 4,
    "drivable area": 5, # Note: YOLO might not detect this category
    "lane": 6           # Note: YOLO might not detect this category
} 
gt_cls = np.array([class_to_id[c] for c in gt_classes if c in class_to_id])

Found 7 ground truth boxes.
Found 7 ground truth classes.
['car', 'car', 'car', 'car', 'truck', 'car', 'car']


In [59]:
pred = results[0]

pred_xyxy = pred.boxes.xyxy.cpu().numpy()
pred_cls = pred.boxes.cls.cpu().numpy().astype(int)
pred_conf = pred.boxes.conf.cpu().numpy()

In [60]:
def iou(boxA, boxB):
    # box: [x1, y1, x2, y2]
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    if boxAArea == 0 or boxBArea == 0:
        return 0

    return interArea / float(boxAArea + boxBArea - interArea)

In [61]:
matched_gt = set()
tp = 0
fp = 0

for i, pbox in enumerate(pred_xyxy):
    pcls = pred_cls[i]

    best_iou = 0
    best_gt = -1

    for j, gbox in enumerate(gt_xyxy):
        if gt_cls[j] != pcls:
            continue
        current_iou = iou(pbox, gbox)
        if current_iou > best_iou:
            best_iou = current_iou
            best_gt = j

    if best_iou >= 0.5:
        if best_gt not in matched_gt:   # avoid double match
            tp += 1
            matched_gt.add(best_gt)
        else:
            fp += 1
    else:
        fp += 1

fn = len(gt_xyxy) - len(matched_gt)


In [62]:
precision = tp / (tp + fp + 1e-9)
recall = tp / (tp + fn + 1e-9)
f1 = 2 * precision * recall / (precision + recall + 1e-9)
accuracy = tp / (tp + fp + fn + 1e-9)

In [63]:
print(precision)
print(recall)

0.0
0.0
